### 1. 데이터 준비

In [8]:
!apt install unzip
!unzip dog.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Archive:  dog.zip
  inflating: train/bo/bo_1.jpg       
  inflating: train/bo/bo_10.jpg      
  inflating: train/bo/bo_11.jpg      
  inflating: train/bo/bo_12.jpg      
 extracting: train/bo/bo_13.jpg      
  inflating: train/bo/bo_14.jpg      
  inflating: train/bo/bo_15.jpg      
  inflating: train/bo/bo_16.jpg      
  inflating: train/bo/bo_17.jpg      
  inflating: train/bo/bo_18.jpg      
  inflating: train/bo/bo_19.jpg      
  inflating: train/bo/bo_2.jpg       
  inflating: train/bo/bo_3.jpg       
  inflating: train/bo/bo_4.jpg       
  inflating: train/bo/bo_5.jpg       
  inflating: train/bo/bo_6.jpg       
  inflating: train/bo/bo_7.jpg       
  inflating: train/bo/bo_8.jpg       
  inflating: train/bo/bo_9.jpg       
  inflating: train/not_bo/1.jpg      
  infl

In [11]:
import numpy as np
import cv2
import glob
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms as T
import torchvision.models as models

In [12]:
transform = T.Compose([T.Resize(256), T.RandomCrop(244), T.ToTensor()])

class Dataset(Dataset):
    def __init__(self, path='train'):
        super(Dataset, self).__init__()

        bo_path = glob.glob(path+'/bo/*.jpg')
        notbo_path = glob.glob(path+'/not_bo/*.jpg')
        self.img_path = bo_path + notbo_path
        self.label_list = [1]*len(bo_path) + [0]*len(notbo_path)

    def __getitem__(self, index):
        img = cv2.imread(self.img_path[index])
        img_pil = Image.fromarray(img)
        self.img_tensor = transform(img_pil)
        self.label_tensor = torch.tensor(self.label_list[index])
        return self.img_tensor.to("cuda:0"), self.label_tensor.to("cuda:0")

    def __len__(self):
        return len(self.img_path)

In [13]:
training_dataset = Dataset('train')
validation_dataset = Dataset('valid')

In [14]:
training_loader = DataLoader(dataset=training_dataset, batch_size=8, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=8, shuffle=False)

### 2. 뉴럴네트워크 모델링

In [27]:
#vgg16 = models.vgg16()
vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 76.2MB/s]


In [28]:
for param in vgg16.parameters():
    param.requires_grad = False

In [29]:
num_features = vgg16.classifier[0].in_features
vgg16.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Linear(256, 2)
)

In [30]:
vgg16 = vgg16.to('cuda:0')

### 3. 손실함수와 최적화기법 정의

In [31]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.0001)

### 3. 뉴럴네트워크 훈련

In [32]:
for epoch in range(10):
    loss_val = 0
    for itr, data in enumerate(training_loader):
        optimizer.zero_grad()

        inputs, labels = data
        pred = vgg16(inputs)
        loss = loss_function(pred, labels)

        loss.backward()

        optimizer.step()

        loss_val += loss.item()
    print("Epoch: ", epoch+1, " , Loss: ", loss_val)

Epoch:  1  , Loss:  8.621671382337809
Epoch:  2  , Loss:  1.7960744048468769
Epoch:  3  , Loss:  0.3592872351873666
Epoch:  4  , Loss:  0.273036266095005
Epoch:  5  , Loss:  0.09288019145606086
Epoch:  6  , Loss:  0.07390306067827623
Epoch:  7  , Loss:  0.05486678182205651
Epoch:  8  , Loss:  0.03361212946037995
Epoch:  9  , Loss:  0.026950100000249222
Epoch:  10  , Loss:  0.023187731254438404


### 5. 성능평가

In [33]:
pred_list = []
label_list = []

for itr, data in enumerate(validation_loader):
    inputs, labels = data

    pred = vgg16(inputs)
    pred_category = torch.argmax(pred, dim=1)

    pred_list = pred_list + list(pred_category.cpu())
    label_list = label_list + list(labels.cpu())

In [34]:
accu = np.mean( np.array(pred_list) == np.array(label_list) )
print("Validation accuracy: ", accu*100)

Validation accuracy:  96.66666666666667
